In [1]:
import os
from PIL import Image, ImageDraw, ImageFont
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import gradio as gr


data_dir = "/Users/zumagalievamaral/Desktop/kaznu_resized"  

class_names = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
n_classes = len(class_names)

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,) * 3, (0.5,) * 3)
])


class KazNU_Dataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.paths, self.labels = [], []
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        for cls in self.classes:
            cls_path = os.path.join(root_dir, cls)
            for img_file in os.listdir(cls_path):
                if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.paths.append(os.path.join(cls_path, img_file))
                    self.labels.append(cls)

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = self.classes.index(self.labels[idx])
        return img, label


dataset = KazNU_Dataset(data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)


class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 4 * 4)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# 5. Оқыту
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomCNN(n_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):  # Уақытты үнемдеу үшін аздап epoch
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


torch.save(model.state_dict(), "custom_cnn.pth")
model.load_state_dict(torch.load("custom_cnn.pth", map_location=device))
model.eval()


def predict_and_annotate(img):
    model.eval()
    image_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image_tensor)
        probs = torch.softmax(output, dim=1)[0].cpu().numpy()

    top5_idx = probs.argsort()[-5:][::-1]
    results = {class_names[i]: float(probs[i]) for i in top5_idx}

    label = class_names[top5_idx[0]]
    confidence = probs[top5_idx[0]]

   
    if confidence < 0.70:
        annotated_img = img.copy().convert("RGB")
        draw = ImageDraw.Draw(annotated_img)
        try:
            font = ImageFont.truetype("arial.ttf", size=20)
        except:
            font = ImageFont.load_default()
        message = "Бұл сурет деректерге сәйкес емес"
        draw.rectangle([0, 0, annotated_img.width, 30], fill=(255, 0, 0))
        draw.text((5, 5), message, fill="white", font=font)
        return annotated_img, {"Хабарлама": message}

   
    annotated_img = img.copy().convert("RGB")
    draw = ImageDraw.Draw(annotated_img)
    try:
        font = ImageFont.truetype("arial.ttf", size=20)
    except:
        font = ImageFont.load_default()
    text = f"{label}: {confidence*100:.1f}%"
    draw.rectangle([0, 0, annotated_img.width, 30], fill=(0, 128, 0))
    draw.text((5, 5), text, fill="white", font=font)

    return annotated_img, results


demo = gr.Interface(
    fn=predict_and_annotate,
    inputs=gr.Image(type="pil", label="Суретті жүктеңіз", sources=["upload"]),
    outputs=[
        gr.Image(label="Нәтиже"),
        gr.Label(label="Топ‑4 ықтималдық")
    ],
    title="Custom CNN: KazNU Ғимаратын тану",
    description="Бұл модель тек 'kaznu_resized' деректеріндегі суреттерге арналған."
)

if __name__ == "__main__":
    demo.launch()

[W NNPACK.cpp:61] Could not initialize NNPACK! Reason: Unsupported hardware.


Epoch 1, Loss: 16.9781
Epoch 2, Loss: 11.8499
Epoch 3, Loss: 8.7930
Epoch 4, Loss: 7.1049
Epoch 5, Loss: 5.8692
Epoch 6, Loss: 5.2585
Epoch 7, Loss: 5.7498
Epoch 8, Loss: 4.5122
Epoch 9, Loss: 4.3600
Epoch 10, Loss: 3.1191
Epoch 11, Loss: 3.0318
Epoch 12, Loss: 4.0888
Epoch 13, Loss: 2.4074
Epoch 14, Loss: 2.4422
Epoch 15, Loss: 2.6223
Epoch 16, Loss: 2.1080
Epoch 17, Loss: 1.2930
Epoch 18, Loss: 1.1804
Epoch 19, Loss: 1.1219
Epoch 20, Loss: 0.7630
* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
